In [106]:
import openai
import pandas as pd
import numpy as np
import json
import os
from scipy import stats
from scipy.stats import chi2_contingency

In [107]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [108]:
Main = pd.read_excel("../UCNets_Classification/Hand_Coding_Surveys/e1b/e1ab_Master.xlsx", engine='openpyxl', sheet_name="Master_coded").drop(["prim_key"], axis=1)
Main = Main.iloc[:400, :20]
Main
#Main.columns = range(Main.shape[1])

bad = pd.read_csv('data/e1b/e1b_bad_categorization_20_cats.csv').drop(['link1','json'], axis=1)
bad = bad.rename(columns = {"survey_response": "Response"})

good = pd.read_csv('data/e1b/e1b_good_categorization_20_cats.csv').drop(['link1','json'], axis=1)
good = good.rename(columns={"survey_response": "Response"})

cot = pd.read_csv('data/e1b/e1b_cot_categorization_20_cats.csv').drop(['link1','json'], axis=1)
cot = cot.rename(columns={"survey_response": "Response"})

cove = pd.read_csv('data/e1b/e1b_cove_categorization_20_cats.csv').drop(['link1','link2','json'], axis=1)
cove = cove.rename(columns={"survey_response": "Response"})

oneshot = pd.read_csv('data/e1b/e1b_1s_categorization_20_cats.csv').drop(['link1','json'], axis=1)
oneshot = oneshot.rename(columns={"survey_response": "Response"})

fewshot = pd.read_csv('data/e1b/e1b_fs_categorization_20_cats.csv').drop(['link1','json'], axis=1)
fewshot = fewshot.rename(columns={"survey_response": "Response"})

Main

,Response,Simple_Assertion,Implicit_Family_Duty,Explicit_Family_Norms,Close_knit,Personal_Character,Past_Support,Not_Reliable,Reciprocal_Support,Estrangement,Independence,Resources,Financial_Resources,Geographic_Proximity,No_Family,Verbal,Duration,Single_Person,Other_Help,Other_no_Help
0,"Probably don't have, because not many relatives",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,"Definitely have, because they love me",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Might have, because they live pretty far away.",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Definitely have, because they love me and woul...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Definitely have, because they've been there fo...",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"Definitely have, because my sister would alway...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
396,"Definitely have, because close fam",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
397,"Definitely have, because i have so many of the...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
398,"Definitely have, because close",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
cove

,Response,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,"probably don't have, because not many relatives",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,"definitely have, because they love me",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"might have, because they live pretty far away.",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,"definitely have, because they love me and woul...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"definitely have, because they've been there fo...",0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"definitely have, because they're my family i k...",0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,"definitely have, because my brother and sister...",0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
397,"definitely have, because we always take care o...",0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
398,"probably don't have, because lack of connectio...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


bad.fillna(0, inplace=True)
bad['Response'] = bad['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
Main['Response'] = Main['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

Main_bad = pd.merge(Main, bad, on = "Response", how = "left")

Main_bad.to_csv('data/e1b/Bad.csv')

good.fillna(0, inplace=True)

good['Response'] = good['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

good = good.astype(int, errors='ignore')

Main_good = pd.merge(Main, good, on="Response", how="left")

Main_good.to_csv('data/e1b/Good.csv')

cot.fillna(0, inplace=True)


cot['Response'] = cot['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)


cot = cot.astype(int, errors='ignore')


Main_cot = pd.merge(Main, cot, on="Response", how="left")

Main_cot.to_csv('data/e1b/COT.csv')

oneshot.fillna(0, inplace=True)

oneshot['Response'] = oneshot['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

oneshot = oneshot.astype(int, errors='ignore')

Main_oneshot = pd.merge(Main, oneshot, on="Response", how="left")

Main_oneshot.to_csv('data/e1b/oneshot.csv')

fewshot.fillna(0, inplace=True)

fewshot['Response'] = fewshot['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

fewshot = fewshot.astype(int, errors='ignore')

Main_fewshot = pd.merge(Main, fewshot, on="Response", how="left")

Main_fewshot.to_csv('data/e1b/fewshot.csv')

fewshot.fillna(0, inplace=True)

cove['Response'] = cove['Response'].str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)

cove = cove.astype(int, errors='ignore')

Main_cove = pd.merge(Main, fewshot, on="Response", how="left")

Main_fewshot.to_csv('data/e1b/cove.csv')

I cleaned this up manually to fill in some gaps that didn't match

In [110]:
bad = pd.read_csv('data/e1b/Bad.csv').drop(['Response'], axis=1)
good = pd.read_csv('data/e1b/Good.csv').drop(['Response'], axis=1)
cot = pd.read_csv('data/e1b/COT.csv').drop(['Response'], axis=1)
cove = pd.read_csv('data/e1b/cove.csv').drop(['Response'], axis=1)
oneshot = pd.read_csv('data/e1b/oneshot.csv').drop(['Response'], axis=1)
fewshot = pd.read_csv('data/e1b/fewshot.csv').drop(['Response'], axis=1)

Main = Main.drop(['Response'], axis=1)

In [111]:
Main

,Simple_Assertion,Implicit_Family_Duty,Explicit_Family_Norms,Close_knit,Personal_Character,Past_Support,Not_Reliable,Reciprocal_Support,Estrangement,Independence,Resources,Financial_Resources,Geographic_Proximity,No_Family,Verbal,Duration,Single_Person,Other_Help,Other_no_Help
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
396,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
398,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
fewshot

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
396,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
397,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
398,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [113]:
bad

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
396,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
397,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
398,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [114]:
print(f"The dataframe Main has {len(Main)} rows.")

print(f"The dataframe bad has {len(bad)} rows.")

print(f"The dataframe good has {len(good)} rows.")

print(f"The dataframe cot has {len(cot)} rows.")

#print(f"The dataframe cove has {len(cove)} rows.")

print(f"The dataframe oneshot has {len(oneshot)} rows.")

print(f"The dataframe fewshot has {len(fewshot)} rows.")


Main.fillna(0, inplace=True)
bad.fillna(0, inplace=True)
good.fillna(0, inplace=True)
cot.fillna(0, inplace=True)
cove.fillna(0, inplace=True)
oneshot.fillna(0, inplace=True)
fewshot.fillna(0, inplace=True)

The dataframe Main has 400 rows.
The dataframe bad has 400 rows.
The dataframe good has 400 rows.
The dataframe cot has 400 rows.
The dataframe oneshot has 400 rows.
The dataframe fewshot has 400 rows.


In [115]:
Main = Main.astype(int, errors='ignore')
bad = bad.astype(int, errors='ignore')
good = good.astype(int, errors='ignore')
cot = cot.astype(int, errors='ignore')
cove = cove.astype(int, errors='ignore')
oneshot = oneshot.astype(int, errors='ignore')
fewshet = fewshot.astype(int, errors='ignore')

In [116]:
Main.head()

,Simple_Assertion,Implicit_Family_Duty,Explicit_Family_Norms,Close_knit,Personal_Character,Past_Support,Not_Reliable,Reciprocal_Support,Estrangement,Independence,Resources,Financial_Resources,Geographic_Proximity,No_Family,Verbal,Duration,Single_Person,Other_Help,Other_no_Help
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
association_measures = []
p_values = []
jaccard_similarities = []
cosine_similarities = []

for i in range(5):
    main_col = Main.columns[i]
    Rebeca_col = Rebeca.columns[i]

    # Create a contingency table
    contingency_table = pd.crosstab(Main[main_col], Rebeca[Rebeca_col])

    # Calculate the Chi-squared test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    association_measures.append(chi2)  # chi2 as a measure of association
    p_values.append(p_value)

    # Calculate Jaccard Similarity
    intersection = sum((Main[main_col] == 1) & (Rebeca[Rebeca_col] == 1))
    union = sum((Main[main_col] == 1) | (Rebeca[Rebeca_col] == 1))
    jaccard_similarity = intersection / union if union != 0 else 0
    jaccard_similarities.append(jaccard_similarity)

Rebeca_df = pd.DataFrame({
    'Chi2_Main_Rebeca': association_measures,
    'P_value_Main_Rebeca': p_values,
    'Jaccard_Similarity': jaccard_similarities,
}, index=Main.columns[:5])

print(Rebeca_df)

                                 Chi2_Main_Rebeca  P_value_Main_Rebeca  \
Religious Community Involvement        349.100127         6.654271e-78   
Local Venue Visitation                 307.872519         6.348733e-69   
Secular Volunteer Service              350.438400         3.401512e-78   
Online Social Networking               343.179009         1.295831e-76   
Workplace Socializing                  345.060762         5.043710e-77   

                                 Jaccard_Similarity  
Religious Community Involvement            0.911765  
Local Venue Visitation                     0.820513  
Secular Volunteer Service                  0.923077  
Online Social Networking                   0.895833  
Workplace Socializing                      0.904762  


In [120]:
category_dfs = {
    'bad': bad,
    'good': good,
    'cot': cot,
    'cove': cove,
    'oneshot': oneshot,
    'fewshot': fewshot
}
result_dfs = {}

for cat_name, cat_df in category_dfs.items():
    association_measures = []
    p_values = []
    jaccard_similarities = []

    for i in range(19):
        main_col = Main.columns[i]
        cat_col = cat_df.columns[i]

            # Create a contingency table
        contingency_table = pd.crosstab(Main[main_col], cat_df[cat_col])

            # Calculate the Chi-squared test
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        association_measures.append(chi2)  # chi2 as a measure of association
        p_values.append(p_value)

            # Calculate Jaccard Similarity
        intersection = sum((Main[main_col] == 1) & (cat_df[cat_col] == 1))
        union = sum((Main[main_col] == 1) | (cat_df[cat_col] == 1))
        jaccard_similarity = intersection / union if union != 0 else 0
        jaccard_similarities.append(jaccard_similarity)

            # Calculate Cosine Similarity
        dot_product = np.dot(Main[main_col], cat_df[cat_col])
        magnitude_main = np.sqrt(np.dot(Main[main_col], Main[main_col]))
        magnitude_cat = np.sqrt(np.dot(cat_df[cat_col], cat_df[cat_col]))

    # Create the DataFrame
    chi_squared_jaccard_cosine_df = pd.DataFrame({
        'Chi2_Main_Cat': association_measures,
        'P_value_Main_Cat': p_values,
        'Jaccard_Similarity': jaccard_similarities,
    }, index=Main.columns[:19])

    # Store the DataFrame in a dictionary with a unique key
    result_dfs[cat_name] = chi_squared_jaccard_cosine_df

In [121]:
result_dfs

{'bad':                        Chi2_Main_Cat  P_value_Main_Cat  Jaccard_Similarity
 Simple_Assertion           21.645942      3.279029e-06            0.172414
 Implicit_Family_Duty      123.374638      1.154552e-28            0.438596
 Explicit_Family_Norms     137.810418      8.017720e-32            0.456522
 Close_knit                103.836732      2.196941e-24            0.533865
 Personal_Character        119.829204      6.894869e-28            0.393939
 Past_Support              207.514213      4.788605e-47            0.619565
 Not_Reliable              128.743341      7.718025e-30            0.437500
 Reciprocal_Support        174.263238      8.671585e-40            0.531646
 Estrangement              236.072890      2.825009e-53            0.666667
 Independence               36.725571      1.359840e-09            0.208333
 Resources                  46.743003      8.093340e-12            0.186047
 Financial_Resources       204.700295      1.968703e-46            0.604651
 Geog

In [122]:
# Initialize an empty dictionary to hold the average Jaccard Similarity for each category
average_jaccard_similarities = {}

# Iterate over each item in result_dfs to calculate the average Jaccard Similarity
for cat_name, df in result_dfs.items():
    average_jaccard_similarities[cat_name] = df['Jaccard_Similarity'].mean()

# Convert the dictionary of averages into a DataFrame
# The keys become the column names, and the values become the row values
average_jaccard_df = pd.DataFrame(average_jaccard_similarities, index=["Average_Jaccard_Similarity"])

print(average_jaccard_df)

                                 bad     good       cot      cove   oneshot  \
Average_Jaccard_Similarity  0.384365  0.41907  0.440361  0.422621  0.415768   

                             fewshot  
Average_Jaccard_Similarity  0.422621  


In [124]:
average_jaccard_df_transposed = average_jaccard_df.T.rename(columns={"Average_Jaccard_Similarity": "Average_Jaccard"})

In [125]:
average_jaccard_df_transposed.to_csv('data/e1b/average_jaccard.csv')

In [126]:
# Initialize an empty DataFrame to hold the Jaccard Similarity columns
jaccard_similarity_df = pd.DataFrame()

# Iterate over the result_dfs dictionary to extract the Jaccard Similarity column
for cat_name, df in result_dfs.items():
    # Add the extracted column to the new DataFrame with a new column name
    jaccard_similarity_df[cat_name] = df['Jaccard_Similarity']

print(jaccard_similarity_df)

                            bad      good       cot      cove   oneshot  \
Simple_Assertion       0.172414  0.093220  0.136986  0.238095  0.139535   
Implicit_Family_Duty   0.438596  0.338235  0.350000  0.369863  0.361446   
Explicit_Family_Norms  0.456522  0.384615  0.547619  0.481481  0.581818   
Close_knit             0.533865  0.688776  0.653846  0.644231  0.626728   
Personal_Character     0.393939  0.437500  0.416667  0.437500  0.358974   
Past_Support           0.619565  0.619565  0.619565  0.623656  0.642105   
Not_Reliable           0.437500  0.625000  0.555556  0.444444  0.529412   
Reciprocal_Support     0.531646  0.650000  0.619048  0.561644  0.634921   
Estrangement           0.666667  0.655172  0.642857  0.642857  0.714286   
Independence           0.208333  0.294118  0.352941  0.357143  0.263158   
Resources              0.186047  0.166667  0.190476  0.318182  0.261905   
Financial_Resources    0.604651  0.682927  0.636364  0.738095  0.659091   
Geographic_Proximity   0.

In [79]:
jaccard_similarity_df.to_csv('data/a19f/similiarity_jaccard.csv')